**Install necessary packages**

In [1]:
!pip install anvil-uplink
!pip install transformers
!pip install docx2txt
!pip install pdfplumber

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Using cached argparse-1.4.0-py2.py3-none-any.whl (23 kB)


Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [2]:
!pip install torch==1.8.2+cu111 torchvision==0.9.2+cu111 torchaudio===0.8.2 -f https://download.pytorch.org/whl/lts/1.8/torch_lts.html

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in links: https://download.pytorch.org/whl/lts/1.8/torch_lts.html


In [ ]:
!pip install transformers
!pip install docx2txt
!pip install anvil-uplink
!pip install pdfplumber
import anvil.server

from bs4 import BeautifulSoup
import docx2txt
from transformers import pipeline

summarizer = pipeline("summarization")

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


**Import packages**

In [ ]:
import anvil.server

from bs4 import BeautifulSoup
import docx2txt
from transformers import pipeline
import requests
import pdfplumber

**Load model**

In [ ]:
summarizer = pipeline("summarization")

**Connet with anvil**

In [ ]:
anvil.server.connect('U5ZALWW3SL2TBJHC6BYXFCSL-7Q36I7VBIV4PEZDG')

**Abstractive summarization**

In [ ]:
@anvil.server.callable
def abstractive_summmary(user_input,percentage):
  max_chunk = 500
  user_input = user_input.replace('.', '.<eos>')
  user_input = user_input.replace('?', '?<eos>')
  user_input = user_input.replace('!', '!<eos>')
  sentences = user_input.split('<eos>')
  words = len(user_input.split(' '))
  current_chunk = 0 
  chunks = []

  for sentence in sentences:
    if len(chunks) == current_chunk + 1: 
        if len(chunks[current_chunk]) + len(sentence.split(' ')) <= max_chunk:
            chunks[current_chunk].extend(sentence.split(' '))
        else:
            current_chunk += 1
            chunks.append(sentence.split(' '))
    else:
        chunks.append(sentence.split(' '))

  for chunk_id in range(len(chunks)):
    chunks[chunk_id] = ' '.join(chunks[chunk_id])
  res = summarizer(chunks, max_length=words//len(chunks) , min_length=(words*percentage//100)//len(chunks), do_sample=False)
  text = ' '.join([summ['summary_text'] for summ in res])
  result = [text,len(user_input.split(' ')),len(text.split(' '))]
  return result  

**Web scraping**

In [ ]:
@anvil.server.callable
def web_page_summary(url):
  r = requests.get(url)
  soup = BeautifulSoup(r.text, 'html.parser')
  results = soup.find_all('p')
  text = [result.text for result in results]
  user_input = ' '.join(text)
  return user_input

**Extract text from DOCX**

In [ ]:
@anvil.server.callable
def docx(media,a):
  with open(f"/tmp/{a}", "wb") as f:
      f.write(media.get_bytes())
  txt_file = docx2txt.process(f"/tmp/{a}")
  txt_file = txt_file.replace('\n','')  
  return txt_file

**Extract text from PDF**

In [ ]:
@anvil.server.callable
def pdf1(media,a):
  with open(f"/tmp/{a}", "wb") as f:
      f.write(media.get_bytes())
  pdfreader = PyPDF2.PdfFileReader(f"/tmp/{a}")
  x = pdfreader.numPages
  text = ""
  for i in range(x):
    pageobj = pdfreader.getPage(i)
    text += pageobj.extractText()
  text1 = text.replace('\n','')
  return text1

In [ ]:
@anvil.server.callable
def pdf(media,a):
  with open(f"/tmp/{a}", "wb") as f:
      f.write(media.get_bytes())
  text = ""
  with pdfplumber.open(f"/tmp/{a}") as pdf:
    no_of_pages = len(pdf.pages)
    for i in range(no_of_pages):
      content = pdf.pages[i].extract_text()
      text += content
      text = text.replace("\n"," ")
    return text
